<a href="https://colab.research.google.com/github/parthnijh/intro-to-pytorch/blob/main/Pytorch_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [2]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

# train test split


In [3]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)


In [4]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [5]:

encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [6]:
import torch
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

# Defining the model

In [20]:
class MySimpleNN():
  def __init__(self,X):
    self.weights=torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)

  def forward(self,X):
    z=torch.matmul(X,self.weights)+self.bias
    y_pred=torch.sigmoid(z)
    return y_pred

  def loss_function(self,ypred,y):
    epsilon=1e-7
    ypred=torch.clamp(ypred,epsilon,1-epsilon)
    loss=-(y_train_tensor*torch.log(ypred)+(1-y_train_tensor)*torch.log(1-ypred)).mean()
    return loss




In [18]:
# Important parameters

learning_rate=0.1
epochs=25

# training a pipeline

In [25]:
#create a model
model=MySimpleNN(X_train_tensor)

#forward pass
#loss calc
#back pass
#update weights and bias
for epoch in range(epochs):
  ypred=model.forward(X_train_tensor)
  loss=model.loss_function(ypred,y_train_tensor)
  loss.backward()
  with torch.no_grad():
    model.weights-=learning_rate*model.weights.grad
    model.bias-=learning_rate*model.bias.grad
  model.weights.grad.zero_()
  model.bias.grad.zero_()
  print(f"Epoch {epoch+1} Loss :{loss.item()}")


Epoch 1 Loss :3.536444098965455
Epoch 2 Loss :3.3985298658043193
Epoch 3 Loss :3.2555271940228354
Epoch 4 Loss :3.106644145531425
Epoch 5 Loss :2.950044293995179
Epoch 6 Loss :2.7904733635857757
Epoch 7 Loss :2.630846838826868
Epoch 8 Loss :2.4626235280995883
Epoch 9 Loss :2.2904990575942197
Epoch 10 Loss :2.1201052227665516
Epoch 11 Loss :1.9477091133154
Epoch 12 Loss :1.7798769554185545
Epoch 13 Loss :1.6172889832417987
Epoch 14 Loss :1.459854129430595
Epoch 15 Loss :1.3159501059153536
Epoch 16 Loss :1.1879241535959082
Epoch 17 Loss :1.0778812277263232
Epoch 18 Loss :0.9872346541825774
Epoch 19 Loss :0.9161630315995685
Epoch 20 Loss :0.8632265898432792
Epoch 21 Loss :0.8254967686047795
Epoch 22 Loss :0.799283553396826
Epoch 23 Loss :0.7810639220185496
Epoch 24 Loss :0.768077664358613
Epoch 25 Loss :0.7584322185335594


# Evaluation

In [27]:
with torch.no_grad():
  ypred=model.forward(X_test_tensor)
  ypred = (ypred > 0.9).float()
  accuracy = (ypred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6608186960220337
